# Character Prediction

## Word to Vector and Bag of Words dataframe creation

In [14]:
import pandas as pd
import numpy as np
import spacy
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Importing Data 
train_df = pd.read_csv('train_df.csv')
val_df = pd.read_csv('val_df.csv')
test_df = pd.read_csv('test_df.csv')


## Word 2 Vector Dataset

In [16]:
# nlp = spacy.load("en_core_web_sm")

# def find_max_length(data):
#     return max(max(len(nlp(dialogue)) for dialogue in set) for set in data)

# max_length = find_max_length([train_df['dialog'], val_df['dialog'], test_df['dialog']])

# def word2vec_seq(data, max_length):
#     # Extracting norm vector values
#     word_vectors = []
#     for dialogue in data['dialog']:
#         tokens = nlp(dialogue)
#         dialogue_vectors = [token.vector_norm for token in tokens]
#         word_vectors.append(dialogue_vectors)

#     # Padding 
#     for i in range(len(word_vectors)):
#         word_vectors[i] += [100] * (max_length - len(word_vectors[i]))

#     df = pd.DataFrame(word_vectors)
#     df.columns = [f"word_{i}" for i in range(1, max_length + 1)]

#     df = pd.concat([data, df], axis=1)
#     return df  

# train_B = word2vec_seq(train_df, max_length)
# val_B = word2vec_seq(val_df, max_length)
# test_B = word2vec_seq(test_df, max_length)

In [17]:
# train_B.to_csv('train_B.csv', index=False)
# val_B.to_csv('val_B.csv', index=False)
# test_B.to_csv('test_B.csv', index=False)

## Bag of Words Dataset

In [18]:

stopwords_set = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")

# Removing; punctuation, stopwords and stemming words
def bow_preprocess(df):
    preprocessed = []
    
    for dialog in df['dialog']:
        tokens = re.sub(r"[^a-zA-Z0-9]", " ", dialog).split()
        alfa = [stemmer.stem(word.lower()) for word in tokens if word not in stopwords_set]
        bravo = str()
        for i in range(len(alfa)):
            bravo += str(alfa[i])
            bravo += ' '
        preprocessed.append(bravo)

    df['dialog'] = preprocessed
    return df


train_C = bow_preprocess(train_df)
val_C = bow_preprocess(val_df)
test_C = bow_preprocess(test_df)

# Creating a set of unique words amongst all dialoges
all_dialogs = pd.concat([train_C['dialog'], val_C['dialog'], test_C['dialog']], ignore_index=True)
wordset = set([word for dialog in all_dialogs for word in dialog.split()])

# Creating dataframe
def BOW_df(wordset, df):

    bow_df = pd.DataFrame(columns=list(wordset))
    
    for i, dialog in enumerate(df['dialog']):
        dialog_tf_diz = {}
        for word in dialog.split():
            if word in wordset:
                if word in dialog_tf_diz:
                    dialog_tf_diz[word] += 1
                else:
                    dialog_tf_diz[word] = 1
        
        bow_df.loc[i] = [dialog_tf_diz.get(word, 0) for word in wordset]
    return pd.concat([df, bow_df], axis=1)

train_C = BOW_df(wordset, train_C)
val_C = BOW_df(wordset, val_C)
test_C = BOW_df(wordset, test_C)

In [19]:
train_C.to_csv('train_C.csv', index=False)
val_C.to_csv('val_C.csv', index=False)
test_C.to_csv('test_C.csv', index=False)

## TF_IDF Dataset

In [20]:


def tf_idf__df(df):
    preprocessed_dialogs = bow_preprocess(df)
    tfidf_vectorizer = TfidfVectorizer(vocabulary=wordset)
    tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_dialogs['dialog'])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=list(wordset))
    return pd.concat([df, tfidf_df], axis=1)

train_D = tf_idf__df(train_df)
val_D = tf_idf__df(val_df)
test_D = tf_idf__df(test_df)

In [21]:
train_D.to_csv('train_D.csv', index=False)
val_D.to_csv('val_D.csv', index=False)
test_D.to_csv('test_D.csv', index=False)